In [ ]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

In [3]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [ ]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

In [13]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=100):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Average reward: -98.0
Success rate: 1/100


This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
# Code goes here
def better_agent_function(observation):
    #calculating the target angle to balance the rocket so that it could land on target
    target_angle = observation[Obs.X.value] * 0.5 + observation[Obs.VX.value] * 0.5

    #calculating the difference between the ideal angle and the actual rocket angle
    angle_error_correction = (target_angle - observation[Obs.ANGLE.value])

    #calculating the ideal thrust
    control_signal = angle_error_correction - observation[Obs.ANGULAR_VELOCITY.value]

    action = Act.NO_OP.value

    #acting based on the control_signal value
    if control_signal > 0.15:
        action = Act.LEFT.value
    elif control_signal < -0.15:
        action = Act.RIGHT.value

    #Decision when rocket angle is not too extreme 
    if abs(observation[Obs.ANGLE.value]) < 0.5:
        if observation[Obs.VY.value] < -0.1:
            action = Act.MAIN.value
    
    #Decision when the left and right leg makes contact on landing surface
    if observation[Obs.LEFT_LEG_CONTACT.value] and observation[Obs.RIGHT_LEG_CONTACT.value]:
        action = Act.NO_OP.value

    return action

better_rewards = run_episodes(better_agent_function)
print(better_rewards)
print(f"Average reward: {np.average(better_rewards)}")
print(f"Success rate: {np.sum(np.array(better_rewards) == 100)}/{len(rewards)}")



[-100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, np.float64(-0.2883425350688771), 100, -100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, np.float64(-0.006137363436720306), 100, 100, 100, 100, -100, 100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, 100, 100]
Average reward: 77.99705520101494
Success rate: 88/100


In [ ]:
#Fungsi yang digunakan untuk menjalankan agent pada satu skenario 
run_episode(better_agent_function)